In [1]:
from html.parser import HTMLParser
from fastcore.all import *
from urllib.parse import *

In [2]:
class HTMLParseAttrs(HTMLParser):
    def reset(self):
        super().reset()
        self.found = set()
    def handle_starttag(self, tag, attrs):
        a = first(v for k,v in attrs if k in ("src","href"))
        if a: self.found.add(a)
    handle_startendtag = handle_starttag

In [3]:
path = Path('../fast.ai/_site/')

In [4]:
def local_url(u, root, host, fname):
    islocal=False
    for o in 'http://','https://','http://www.','https://www.':
        if u.startswith(o+host): u,islocal = remove_prefix(u, o+host),True
    fpath = Path(fname).parent
    p = list(urlparse(u))[:3]+['','','']
    if islocal or (not p[0] and not p[1]):
        u = p[2]
        if u and u[0]=='/': return (root/u[1:]).resolve()
        else: return (fpath/u).resolve()
    if not p[0]: p[0]='http'
    if p[0] not in ('http','https'): return ''
    return urlunparse(p)

In [5]:
from collections import defaultdict

In [6]:
def get_links(path, host):
    fns = L(path.glob('**/*.html'))+L(path.glob('**/*.htm'))
    h = HTMLParseAttrs()
    found = {}
    for fn in fns:
        h.reset()
        fn = fn.resolve()
        h.feed(fn.read_text())
        found[fn] = L(h.found)
    res = defaultdict(list)
    for fn,links in found.items():
        for link in links: res[local_url(link, root=path, host=host, fname=fn)].append(fn)
    return res

In [7]:
l = get_links(path, host='fast.ai')

In [8]:
files = {k:v for k,v in l.items() if isinstance(k,Path) and not k.exists()}

In [9]:
files

{}